## Database Engineering

SQLAlchemy to model table schemas and create a sqlite database with loaded panas tables.

### CSV file table loading 

In [1]:
# Dependencies to import data csv files 
import pandas as pd 

In [2]:
# load Data to be added to the database later 
measurements=pd.read_csv('data/clean_measurements.csv')
stations=pd.read_csv('data/hawaii_stations.csv')

In [3]:
measurements=measurements.rename(columns = {'Unnamed: 0':'id'})
measurements.id=measurements.id + 1
measurements.head()

,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65
1,2,USC00519397,2010-01-02,0.00,63
2,3,USC00519397,2010-01-03,0.00,74
3,4,USC00519397,2010-01-04,0.00,76
4,5,USC00519397,2010-01-06,0.03,73


In [36]:
measurements.shape

(19550, 4)

In [38]:
len(measurements.date.unique())

2792

In [42]:
len(measurements.id.unique())

19550

In [32]:
stations.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


### Create empty database 

In [4]:
# Dependencies to create empty database
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite") # database created with zero size 
Base = declarative_base()
Base.metadata.create_all(engine)

# check in the folder if the empty hawaii.sqlite is created 

### Create Classes for the database Tables  

In [6]:
# Dependencies to create classes 
from sqlalchemy import Column, Float, Integer, String

In [7]:
# make sure you select the proper primary_key and match the loaded table cols 
class Measurements(Base): # for the measuremnts table 
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)

In [8]:
class Stations(Base): # for the stations table
    __tablename__ = 'stations'

    station = Column(String, primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [9]:
# Push empty tables from the classes just created on the Base
Base.metadata.create_all(engine)
# QC open the data base to check if created 

### Preper Data to be pushed to database 

In [10]:
#transofrm dataframes to dicts as list of record dicts  
measurements_data = measurements.to_dict(orient='records')
stations_data = stations.to_dict(orient='records')

In [11]:
measurements_data

[{'date': '2010-01-01',
  'id': 1,
  'prcp': 0.08,
  'station': 'USC00519397',
  'tobs': 65},
 {'date': '2010-01-02',
  'id': 2,
  'prcp': 0.0,
  'station': 'USC00519397',
  'tobs': 63},
 {'date': '2010-01-03',
  'id': 3,
  'prcp': 0.0,
  'station': 'USC00519397',
  'tobs': 74},
 {'date': '2010-01-04',
  'id': 4,
  'prcp': 0.0,
  'station': 'USC00519397',
  'tobs': 76},
 {'date': '2010-01-06',
  'id': 5,
  'prcp': 0.03,
  'station': 'USC00519397',
  'tobs': 73},
 {'date': '2010-01-07',
  'id': 6,
  'prcp': 0.06,
  'station': 'USC00519397',
  'tobs': 70},
 {'date': '2010-01-08',
  'id': 7,
  'prcp': 0.0,
  'station': 'USC00519397',
  'tobs': 64},
 {'date': '2010-01-09',
  'id': 8,
  'prcp': 0.0,
  'station': 'USC00519397',
  'tobs': 68},
 {'date': '2010-01-10',
  'id': 9,
  'prcp': 0.0,
  'station': 'USC00519397',
  'tobs': 73},
 {'date': '2010-01-11',
  'id': 10,
  'prcp': 0.01,
  'station': 'USC00519397',
  'tobs': 64},
 {'date': '2010-01-12',
  'id': 11,
  'prcp': 0.0,
  'station': '

In [12]:
stations_data

[{'elevation': 3.0,
  'latitude': 21.2716,
  'longitude': -157.8168,
  'name': 'WAIKIKI 717.2, HI US',
  'station': 'USC00519397'},
 {'elevation': 14.6,
  'latitude': 21.4234,
  'longitude': -157.8015,
  'name': 'KANEOHE 838.1, HI US',
  'station': 'USC00513117'},
 {'elevation': 7.0,
  'latitude': 21.5213,
  'longitude': -157.8374,
  'name': 'KUALOA RANCH HEADQUARTERS 886.9, HI US',
  'station': 'USC00514830'},
 {'elevation': 11.9,
  'latitude': 21.3934,
  'longitude': -157.9751,
  'name': 'PEARL CITY, HI US',
  'station': 'USC00517948'},
 {'elevation': 306.6,
  'latitude': 21.4992,
  'longitude': -158.0111,
  'name': 'UPPER WAHIAWA 874.3, HI US',
  'station': 'USC00518838'},
 {'elevation': 19.5,
  'latitude': 21.33556,
  'longitude': -157.71139,
  'name': 'WAIMANALO EXPERIMENTAL FARM, HI US',
  'station': 'USC00519523'},
 {'elevation': 32.9,
  'latitude': 21.45167,
  'longitude': -157.84888999999998,
  'name': 'WAIHEE 837.5, HI US',
  'station': 'USC00519281'},
 {'elevation': 0.9,
  '

In [13]:
# Use MetaData from SQLAlchemy to reflect the tables
from sqlalchemy import MetaData
metadata = MetaData(bind=engine)
metadata.reflect()

In [14]:
metadata

MetaData(bind=Engine(sqlite:///hawaii.sqlite))

In [15]:
# Save the reference to the proper tables as a variables 
# For bulk upload data, rather than add through individual objects
import sqlalchemy
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [16]:
measurements_table

Table('measurements', MetaData(bind=Engine(sqlite:///hawaii.sqlite)), Column('id', INTEGER(), table=<measurements>, primary_key=True, nullable=False), Column('station', VARCHAR(), table=<measurements>), Column('date', VARCHAR(), table=<measurements>), Column('prcp', FLOAT(), table=<measurements>), Column('tobs', INTEGER(), table=<measurements>), schema=None)

In [17]:
stations_table

Table('stations', MetaData(bind=Engine(sqlite:///hawaii.sqlite)), Column('station', VARCHAR(), table=<stations>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<stations>), Column('latitude', FLOAT(), table=<stations>), Column('longitude', FLOAT(), table=<stations>), Column('elevation', FLOAT(), table=<stations>), schema=None)

In [18]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.

conn = engine.connect()

conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [19]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(measurements_table.insert(), measurements_data)
conn.execute(stations_table.insert(), stations_data)

In [ ]:
# check if tables are updated with data in the Database file 